In [29]:
import pandas as pd
from pandas.io.json import json_normalize
import requests # library to handle requests
from bs4 import BeautifulSoup
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import folium

In [47]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
html = response.text

In [50]:
parsedHtml = BeautifulSoup(html, "html.parser")
# Or table has index 0
tableHtml =parsedHtml.findAll('table')[0]
#Convert table to dataframe
torNbDf = pd.read_html(str(tableHtml))[0]
torNbDf

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [51]:
#Complete Not assigned values in second column with values in third column
for i, r in torNbDf.iterrows():
    if r[2] == 'Not assigned':
        r[2] = r[1]
    else:
        pass

In [52]:
#Remove useless rows
torNbDf.replace('Not assigned', np.nan, inplace=True)
torNbDf.dropna(axis = 0, inplace=True)
torNbDf

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


In [53]:
# Rename PostCode column
torNbDf.rename(columns={'Postcode':'Postal Code'}, inplace=True)
torNbDf.describe()

,Postal Code,Borough,Neighbourhood
count,210,210,210
unique,103,11,207
top,M8Y,Etobicoke,St. James Town
freq,8,44,2


In [55]:
#Remove duplicates
torNbDf = torNbDf.groupby(by = ['Postal Code', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

In [58]:
torNbDf.describe

<bound method NDFrame.describe of     Postal Code      Borough  \
0           M1B  Scarborough   
1           M1C  Scarborough   
2           M1E  Scarborough   
3           M1G  Scarborough   
4           M1H  Scarborough   
..          ...          ...   
98          M9N         York   
99          M9P    Etobicoke   
100         M9R    Etobicoke   
101         M9V    Etobicoke   
102         M9W    Etobicoke   

                                         Neighbourhood  
0                                       Rouge, Malvern  
1               Highland Creek, Rouge Hill, Port Union  
2                    Guildwood, Morningside, West Hill  
3                                               Woburn  
4                                            Cedarbrae  
..                                                 ...  
98                                              Weston  
99                                           Westmount  
100  Kingsview Village, Martin Grove Gardens, Richv...  
101  Albion

In [54]:

# Append GEO columns to our dataframe
#column_names = ['Latitude', 'Longitude'] 
#geo_df = pd.DataFrame(columns=column_names)
#geo_df.head()
#torNbDf = torNbDf.join(geo_df)
#torNbDf.head()

## Complete the coordinates
#geolocator = Nominatim(user_agent="toronto_explorer")
#for index, row in torNbDf.iterrows():
#    neighborhood_name = row['Neighborhood']
#    address = neighborhood_name + ",Toronto"
#    location = geolocator.geocode(address)
#    torNbDf.at[index, 'Latitude'] = location.latitude
#    torNbDf.at[index, 'Longitude'] = location.longitude
#    print(index, neighborhood_name)
#    time.sleep(1)
    
#torNbDf.head()

# I couldn't retireve coordinates from Nominatim, neither from Geocoders...
# I Uploaded a csv table with the information, though the URL https://cocl.us/Geospatial_data doesn't work (503 Service Unavailable)
geo_df = pd.read_csv('https://raw.githubusercontent.com/pangodream/Coursera_Capstone/master/TorontoNeighborhoods/TorontoPCLatLon.csv')

In [22]:
geo_df.head()

,PostalCode,Lat,Lon
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [59]:
geo_df.rename(columns={'PostalCode':'Postal Code'}, inplace=True)
geo_df

,Postal Code,Lat,Lon
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [60]:
#Combine both dataframes
torPcGeo_df = pd.merge(torNbDf, geo_df, on="Postal Code")
torPcGeo_df

,Postal Code,Borough,Neighbourhood,Lat,Lon
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


### GEO and Foursquare

In [61]:
# Toronto coordinates
addr = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(addr)
torontoLat = location.latitude
torontoLon = location.longitude
print('Toronto Coordinates are:'.format(torontoLat, torontoLon))

Toronto Coordinates are:


In [62]:
#I am not being able (AGAIN) to access Nominatim!!!
torontoLat = 43.653963
torontoLon = -79.387207

In [63]:
# Get distinct boroughs
bor_lst = torPcGeo_df['Borough'].unique()
bor_lst

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [81]:
# Foursquare
CLIENT_ID = "****"
CLIENT_SECRET = "*****"
VERSION = '20190722' 
LIMIT = 100

#Defining a funtion to retrieve near venues
def getNearbyVenues(postcode, latitudes, longitudes, radius=500):
    exitCounter = 0;
    venues_list=[]
    for name, lat, lng in zip(postcode, latitudes, longitudes):      
        exitCounter = exitCounter + 1;
        #Added this to limit number of requests during tests
        #if exitCounter == 3:
        #   break
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
torVenues = getNearbyVenues(torPcGeo_df['Neighbourhood'], torPcGeo_df['Lat'], torPcGeo_df['Lon'])
torVenues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [83]:
#Transform venue categories by One Hot encoding
venue_dummies = pd.get_dummies(torVenues[['Venue Category']], prefix="", prefix_sep="")
venue_dummies['Neighborhood']=torVenues['Neighborhood']
venue_dummies

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2209,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2210,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
#Grouping by Neighbourhood calculate frequencies of venues
torVenFreq = venue_dummies.groupby('Neighborhood').mean().reset_index()
torVenFreq.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [86]:
#Clustering
from sklearn.metrics import silhouette_score
torForClustering = torVenFreq.drop('Neighborhood', axis = 1)
for i in range(2, 10):
    kmeans = KMeans(n_clusters=i, random_state=0).fit(torForClustering)
    label = kmeans.labels_
    sil_coeff = silhouette_score(torForClustering, label, metric='euclidean')
    print(i, sil_coeff)


2 0.21556196568532018
3 0.2819600355435323
4 0.2908245921296329
5 0.29572455636545325
6 0.30157889743657157
7 0.10651837117488455
8 0.25395961970932496
9 0.1320345709454026


In [88]:
#6 seems t obe the best K index
kindex = 6
torForClustering = torVenFreq.drop('Neighborhood', axis = 1)
kmeans = KMeans(n_clusters=kindex, random_state=0).fit(torForClustering)
kmeans.labels_


array([0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 5, 0, 5, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 5, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0], dtype=int32)

In [89]:
torVenFreq['Cluster']=kmeans.labels_
torVenFreq

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Cluster
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,5
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0
97,"Woodbine Gardens, Parkview Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0


In [91]:
torPcGeo_df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
torFinal = pd.merge(torPcGeo_df, torVenFreq, on="Neighborhood")

In [92]:
torFinal

,Postal Code,Borough,Neighborhood,Lat,Lon,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Cluster
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,M9N,York,Weston,43.706876,-79.518188,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
97,M9P,Etobicoke,Westmount,43.696319,-79.532242,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
98,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
99,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## Maps

In [93]:
map_clusters = folium.Map(location=[torontoLat, torontoLon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kindex)
ys = [i + x + (i*x)**2 for i in range(kindex)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torFinal['Lat'], torFinal['Lon'], torFinal['Neighborhood'], torFinal['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters